# Image and Video Processing with Python

In [7]:
import cv2

OpenCV is basically a library for **Computer Vision** in python.
## Loading, Displaying, Resizing and Creating Images

`cv2.imread()` takes, along with file location, a parameter that expects $0$ , $1$ or $-1$.

$0$ = black and white or grayscales

$1$ = rbg image (Yes, python follows *rbg* instead of *rgb*, as if there wasn't enough idiotics)

$-1$ = Color image with *alpha* channel which controls transparency.
### Opening images

In [8]:
img = cv2.imread("..\Files\galaxy.jpg", 0)
print("Dimensions = ",img.ndim)
print(img)  
print(type(img))
print(img.shape)

Dimensions =  2
[[14 18 14 ... 20 15 16]
 [12 16 12 ... 20 15 17]
 [12 13 16 ... 14 24 21]
 ...
 [ 0  0  0 ...  5  8 14]
 [ 0  0  0 ...  2  3  9]
 [ 1  1  1 ...  1  1  3]]
<class 'numpy.ndarray'>
(1485, 990)


So this is a `2-D` matrix, *ndarray* to be precise. And it has a resolution of $1485 X 990$ 

In [9]:
img_c = cv2.imread("..\Files\galaxy.jpg", 1)

### Displaying Images

In [10]:
cv2.imshow("Galaxy", img)
cv2.waitKey(2000)    #Time for the window to close. 0 = close on press of any button
cv2.destroyAllWindows()   #What to do after the aforementioned time

See it was closed after 2 secs.

Now we need to resize the image so that it may fit our screen (or whatever your reason is for resizing.)

### Resizing images

In [11]:
resized_img = cv2.resize(img, (1920, 1080))
cv2.imshow("Galaxy", resized_img)
cv2.waitKey(0)    #Time for the window to close. 0 = close on press of any button
cv2.destroyAllWindows()

See, it was resized. What is happening here is that *python* is resizing the numpy array. It takes the array and creates a new array of required shape by interpolating new data to fit the new size. It does a good job though the expected wierdness like streatching effect will creep into the image.


To maintain the aspect ration of the image, we can do this:

In [12]:
resized_immg_2 = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
cv2.imshow("Galaxy_resized", resized_immg_2)
cv2.waitKey(0)    #Time for the window to close. 0 = close on press of any button
cv2.destroyAllWindows()
print(resized_immg_2.shape)

(742, 495)


So that halved the size (and resolution)

### Writing images (Saving)
As expected the method for this is **`imwrite()`** method

In [13]:
cv2.imwrite("..\Files\P_Galaxy_resized.jpg", resized_immg_2)

True

## Batch image resizing
To batch processing, we need to create a list containing the image file paths. Then we can easily iterate over it.

In [19]:

import glob  # Finds path names given a certain patterns

images= glob.glob(r"..\Files\*.jpg")
i = 0
for image in images:
    img=cv2.imread(image,0)
    re=cv2.resize(img,(200,200))
    cv2.imshow("Hey",re)
    cv2.waitKey(10)
    cv2.destroyAllWindows()
    print(cv2.imwrite(r"..\Files\Batch_res_ex\resized_"+str(i)+".jpg",re), image)
    i += 1

True ..\Files\galaxy.jpg
True ..\Files\kangaroos-rain-australia_71370_990x742.jpg
True ..\Files\Lighthouse.jpg
True ..\Files\Moon sinking, sun rising.jpg
True ..\Files\news.jpg
True ..\Files\photo.jpg
True ..\Files\P_Galaxy_resized.jpg


## Detecting Faces in images
This basically works by using **Cascades** which are some *XML* files that contains the info about the features that an image of a face contains. These features include (but aren't limited to) ratio of the shadows of the eyes and nose and lips. All these features (pixel intensities) are recorded in the XML file using some training samples.

### Haar Cascades
**So what is Haar Cascade?** It is an Object Detection Algorithm used to identify faces in an image or a real time video. The algorithm uses edge or line detection features proposed by Viola and Jones in their research paper “Rapid Object Detection using a Boosted Cascade of Simple Features” published in 2001. The algorithm is given a lot of positive images consisting of faces, and a lot of negative images not consisting of any face to train on them.

For more details on haar cascades visit : https://towardsdatascience.com/face-detection-with-haar-cascade-727f68dafd08

What we'll do is load an image and then pass the model (the XML file) to python. And python will scan the image using `open CV` to search for faces using the model

In [44]:
#face_casc = cv2.CascadeClassifier("Img_n_Cam\Files\haarcascade_frontalface_default.xml")
face_casc = cv2.CascadeClassifier(r'..\Files\haarcascade_frontalface_default.xml')

Here we created a cascade classifier object, which we'll use to classify whether something is a face or not.

One interesting thing is using grayscales image makes the classification much more simpler and accurate as there won't be too many unnecessary information in the image and the classification will be simpler and faster. 
Too many unnecessary info can lead `openCV` to miss out some features due to too much noise or take some random noise to be features pertaining to faces.  

In [45]:
image = cv2.imread(r"..\Files\photo.jpg") 
gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Gray",gray_img)
cv2.waitKey(100)
cv2.destroyAllWindows()

So now we have the orignal colored image and a grayscales version of the image.

Now we'll use **`detectMultiScale`** which will search for *cascadeClassifier* (the frontal face XML) in our image and will return the coordinates of the face in the image (The top left point in terms of pixel and the height and width of the face)

**PARAMETERS:**

**scaleFactor** – Parameter specifying how much the image size is reduced at each image scale.

What this tells is to reduce the image size by a a certain factor so that larger objects can be recognized (the scan box remains of the same size)

1.05 is a good possible value for this, which means you use a small step for resizing, i.e. reduce the size by 5%, you increase the chance of a matching size with the model for detection is found. This also means that the algorithm works slower since it is more thorough.

**minNeighbors** - Parameter specifying how many neighbors each candidate rectangle should have to retain it.

This parameter will affect the quality of the detected faces. Higher value results in fewer detections but with higher quality. 3~6 is a good value for it.

**minSize** - Minimum possible object size. Objects smaller than that are ignored.

**maxSize** – Maximum possible object size. Objects bigger than this are ignored.


In [46]:
faces = face_casc.detectMultiScale(gray_img, 
scaleFactor = 1.05, 
minNeighbors = 5)

print(faces)
print(type(faces))

[[157  84 379 379]]
<class 'numpy.ndarray'>


So the result is a *ndarray* which starts at $(157, 84)$ and is $349 X 349$

Lets draw a rectangle arond the detected face

In [47]:
for x, y, w, h in faces:
    image = cv2.rectangle(image, (x,y), (x+w, y+h), (255, 0, 0), 3)

The parameters of **`cv2.rectangle`** are:
- *The Image object*
- The top left coordinate of the rectangle
- The lower right coordinate of the rectangle
- The color of the boundary of the rectangle
- The width of the boundary

In [49]:
cv2.imshow("Image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(r"..\Files\Det_face.jpg", image)

True

A more challenging detection problem is the *news picture*

In [62]:
img_dif = cv2.imread(r"..\Files\news.jpg")
img_gray = cv2.cvtColor(img_dif, cv2.COLOR_BGR2GRAY)
cv2.imshow("GRAY", img_gray)
cv2.waitKey(200)
cv2.destroyAllWindows()

In [56]:
faces = face_casc.detectMultiScale(img_gray, 
scaleFactor= 1.05,
minNeighbors= 5)
print(faces)

[[ 42 220 108 108]
 [305 379  84  84]]


In [58]:
for x, y, w, h in faces:
    img_dif = cv2.rectangle(img_dif, (x,y), (x+w, y+h), (255, 50, 0), 3)

cv2.imshow("Result", img_dif)
cv2.waitKey(0)
cv2.destroyAllWindows()

See what happened. It wasn't able to detect the man's face and the face in the news paper in his hand. But it also miss-detected the hand of the man as a face

In [60]:
faces = face_casc.detectMultiScale(img_gray, 
scaleFactor= 1.1,
minNeighbors= 5)
print(faces)

[[ 46 220 111 111]]


In [63]:
for x, y, w, h in faces:
    img_dif = cv2.rectangle(img_dif, (x,y), (x+w, y+h), (255, 50, 0), 3)

cv2.imshow("Result", img_dif)
cv2.waitKey(0)
cv2.destroyAllWindows()

So increasing the scale factor helped us to stop the classifier from classifying the hand as a face. But we couldn't detect the two faces.

In [64]:
cv2.imwrite(r"../Files/Det_diffimg.jpg", img_dif)

True

# Capturing video from webcams
Obviously we'll be using `OpenCV` as videos are basically several still frames stacked together.

First we need to read a video. Then we can do processing on the frames.
This implies that first we need to create a **`Video Object`**
The `cv2.VideoCapture()` method triggers a *video capture object* which needs a number (which implies capturing a video from a cameras indexed by the int) or a *path* (which implies capturing from an existing file)

We also need to make sure that we'll release the camera once the job is done. So add the `release` statement in the end while creating the *video capture object*

In [65]:
video = cv2.VideoCapture(0)


video.release()

So, what happened?

Obviously nothing. The camera started and was released immediately.

So lets give it a few seconds so that we can see something happening.

In [68]:
import time

video = cv2.VideoCapture(0)
time.sleep(10)
video.release()

Still don't understand whats happening.

Obviously we aren't showing anything so it isn't returning anything.

We need to create a **`Frame`** object first which will read the images of the *video capture object*

**Check** - boolean

To check whether things are working or not.

**frame** - ndarray representing the image

The ndarray that is printed is the first image.

In [81]:
video = cv2.VideoCapture(0)

check, frame = video.read()  #capture a frame

print(check)
print(frame)

time.sleep(3)     #wait for 3 seconds

cv2.imshow("Capturing", frame)  #show the frame
cv2.waitKey(0)    #wait for user input

video.release()    #release the videocapture object

cv2.destroyAllWindows()   #close all the windows

print(type(frame))

True
[[[151 139 121]
  [149 137 120]
  [154 141 126]
  ...
  [157 158 156]
  [156 158 156]
  [159 162 159]]

 [[143 130 112]
  [148 136 119]
  [156 143 128]
  ...
  [160 161 159]
  [160 161 159]
  [162 164 161]]

 [[147 133 117]
  [160 146 130]
  [165 152 137]
  ...
  [164 164 161]
  [164 164 161]
  [166 166 163]]

 ...

 [[230 204 187]
  [242 218 195]
  [253 233 199]
  ...
  [144 160 160]
  [148 164 164]
  [150 166 165]]

 [[243 215 195]
  [250 224 198]
  [249 226 190]
  ...
  [145 161 161]
  [150 166 165]
  [145 161 160]]

 [[249 220 199]
  [250 223 196]
  [239 216 178]
  ...
  [146 162 161]
  [148 164 163]
  [137 153 152]]]
<class 'numpy.ndarray'>


Lets check out the image (the very first frame of the video).

So our script captured the very first frame of the video with the `video.read` function then it slept (did nothing) for the next few seconds then waited (`waitKey`) for the user input then released the video capture and destroyed the window

In [77]:
cv2.imshow("Cam_cap",frame)
cv2.waitKey(10)
cv2.destroyAllWindows()

We'll use the `frame` object, which is a $3-D$ `ndarrray`, to loop through it and use the `imshow` method to display the frames of the video being captured. => Displaying a video not an still image

But for this we also need to keep changing the frame in the window or it'll be something is a very wierd slid show. So we'll make a few changes.

The implementation has one thing missing so it will crash python needing you to restart the kernel. We'll see what it is and then correct it in the next implementation.

In [1]:
import cv2

video = cv2.VideoCapture(0)

while True:
    check, frame = video.read()  #capture a frame

    print(check)      
    #print(frame)    # No need to print the ndarrays of the frame

    #time.sleep(3)     # no need to wait for 3 seconds   

    cv2.imshow("Capturing", frame)  #show the frame
    cv2.waitKey(100)    #And show it for 0.1 second and then continue the loop
    # Notice we didn't destroy the window so that the frames are displayed properly


video.release()    #release the videocapture object

cv2.destroyAllWindows()   #close all the windows

print(type(frame))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

But as you can see we have no way of coming out of the video capturing (`while` loop). This crashes python. So, let's get over it.

In [6]:
import cv2

video = cv2.VideoCapture(0)

no_of_frames = 0
while True:
    no_of_frames += 1

    check, frame = video.read()  #capture a frame

    print(check)      
    #print(frame)    # No need to print the ndarrays of the frame

    #time.sleep(3)     # no need to wait for 3 seconds   
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Capturing", gray)  #show the frame
    
    key =  cv2.waitKey(5)    #And show it for 0.1 second and then continue the loop
    # Notice we didn't destroy the window so that the frames are displayed properly

    print(frame)

    if key == ord('q'):    #If q is pressed on the keyboard it will break the loop
        break

video.release()    #release the videocapture object

cv2.destroyAllWindows()   #close all the windows

print("Number of frames =", no_of_frames)
print(type(frame))

True
[[[17 10  1]
  [19 11  2]
  [24 11  3]
  ...
  [22 10  0]
  [22 10  0]
  [21  9  0]]

 [[15  9  0]
  [18 10  1]
  [22 10  2]
  ...
  [22 10  0]
  [23 10  0]
  [22  9  0]]

 [[15  9  0]
  [17  9  0]
  [21  9  1]
  ...
  [24  9  0]
  [24  9  0]
  [24  8  0]]

 ...

 [[19  9  2]
  [19  9  2]
  [19  9  2]
  ...
  [21  7  1]
  [20  7  1]
  [20  9  2]]

 [[19  9  2]
  [19  9  2]
  [20  9  2]
  ...
  [20  9  2]
  [20  9  2]
  [19  9  2]]

 [[19  9  2]
  [19  9  2]
  [20  9  2]
  ...
  [21 10  3]
  [20 10  3]
  [19 10  3]]]
True
[[[31  7  1]
  [30  7  1]
  [26  7  0]
  ...
  [28  8  3]
  [28  8  3]
  [29  8  3]]

 [[33  8  2]
  [32  8  2]
  [28  8  1]
  ...
  [27  8  3]
  [28  8  3]
  [28  8  3]]

 [[32  8  2]
  [32  8  2]
  [30  8  3]
  ...
  [25  9  3]
  [25  8  2]
  [26  7  2]]

 ...

 [[17  7  0]
  [17  8  1]
  [16  9  2]
  ...
  [20  8  0]
  [20  8  0]
  [20  8  0]]

 [[16  8  2]
  [16  8  1]
  [17 10  2]
  ...
  [22  9  1]
  [22  8  0]
  [22  8  0]]

 [[16  8  2]
  [17  9  2]
  [17 

So now we have a way to stop the video capturing.

Now we have an idea of how to do motion detection, object detection and similar stuff.